# Parametrisation of gemmae contours


## Documentation and Imports


Created on 27-05-2021

Author: Valentin Laplaud

This code loads gemmae contours from ShapeComputation.ipynb and express them as a function of a curvilinar abcisse. Adding landmark points (notches and attach points) then allows a deformation of the contour to match the position of landmark points.

In [ ]:
## Clean up before script start 

for element in dir():
    if element[0:2] != "__":
        del globals()[element]
import gc
gc.collect()



print('\033[1m' + '\033[4m' + '\nRunning ''ShapeParametrisationNB'' \n' + '\033[0m')

# plotting stuff
import matplotlib as mpl
%matplotlib inline
mpl.use('TkAgg')
%matplotlib inline

COLOR = 'white'
COLOR2 = 'black'

mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams['axes.edgecolor'] = COLOR

mpl.rcParams["figure.facecolor"] = COLOR2
mpl.rcParams["axes.facecolor"] = COLOR2
mpl.rcParams["savefig.facecolor"] = COLOR2
mpl.rcParams['axes.facecolor'] = COLOR2

from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import matplotlib.path as mpltPath
from cycler import cycler
#Default colors
colorcycle = [plt.get_cmap('gist_rainbow')(1. * i/30) for i in range(30)]
mpl.rcParams['axes.prop_cycle'] = cycler(color=colorcycle)

# numbers handling
import numpy as np
import numpy.matlib as mtl
import pandas as pd
    

# signal processing 
from scipy.signal import savgol_filter, correlate, correlation_lags, find_peaks
from scipy.interpolate import interp1d, UnivariateSpline
from scipy import optimize

# images handling
from skimage import io
from skimage.filters import threshold_otsu, gaussian, laplace, sobel
from skimage.measure import label, regionprops, regionprops_table
from skimage.util import invert
from skimage.morphology import binary_opening, binary_closing, remove_small_holes,binary_erosion
from skimage.color import rgb2gray
from skimage.segmentation import active_contour, morphological_geodesic_active_contour,morphological_chan_vese, checkerboard_level_set, inverse_gaussian_gradient
import cv2 as cv

# to hide known warnings
import warnings
warnings.filterwarnings("ignore")

# General system functions
import os
import shutil
import sys

import time

# my functions
sys.path.append('../')
import VallapFunc as vf



##  Define analysis functions

### Getting landmark points 
User input is used to identify three landmarks points on the first immage (attach point and two notches). Then a curvature computation is used on every subsequent image to get points of high inwards curvature. The position of the three landmarks in time is then propagated from the first image to the nearest point of high curvature.

#### Identifiying contour points from user input.

In [ ]:
def getContourPointsCoordinates(P,s,npts,nimg,X,Y,Xc,Yc,Xold,Yold,Xcold,Ycold):            
        
    RGBimg = io.imread(P + '\\' + s + '.tif', key = nimg) # get the  image from tiff stack

    # ask user to click
    %matplotlib qt
    plt.figure(dpi=250)
    plt.imshow(RGBimg)
    plt.plot(X,Y,'c',lw = 0.7)
    plt.plot(Xold,Yold,'b--',lw=0.3)
    plt.plot(Xcold,Ycold,'mo',ms = 2)
    plt.plot(Xc,Yc,'r+',ms = 1.5)
    pts = np.asarray(plt.ginput(npts, timeout=-1))
    plt.close()
    %matplotlib inline

    Points = pts[:]

    for i in range(npts):
        x = pts[i][0]
        y = pts[i][1]

        xc = Xc[np.argmin(np.sqrt(np.square(Xc-x)+np.square(Yc-y)))]            
        yc = Yc[np.argmin(np.sqrt(np.square(Xc-x)+np.square(Yc-y)))]

        Points[i][0] = xc
        Points[i][1] = yc

    return(Points)
 

#### Computing contour curvature unsing circle fitting

In [ ]:
def fitCircle(X,Y):
    
    x_m = np.mean(X)
    y_m = np.mean(Y)    
    
    def calc_R(xc, yc):

        return np.sqrt((X - xc) ** 2 + (Y - yc) ** 2)
    
    def f_2(c):
        
        Ri = calc_R(*c)
        return Ri - Ri.mean()
    
    center_estimate = x_m, y_m
    center_fit, _ = optimize.leastsq(f_2, center_estimate)

    xc_fit, yc_fit = center_fit
    Ri_fit       = calc_R(xc_fit, yc_fit)
    
     #Fitting the radius of the circle
    R_fit        = Ri_fit.mean()
   
    
    if False:
        fig,ax = plt.subplots(dpi=250, facecolor = 'black')
        ax.plot(X,Y,'ro')

        xcircle = xc_fit + R_fit*cos(np.linspace(-pi,pi,100))
        ycircle = yc_fit + R_fit*sin(np.linspace(-pi,pi,100))
        ax.plot(xcircle,ycircle,'k--')

    return([xc_fit, yc_fit,R_fit])
    
def getContourCurvature(X,Y,step):

    newX = np.concatenate((X[-step:],X,X[:step]))    
    newY = np.concatenate((Y[-step:],Y,Y[:step]))
    
    Curv = []
    xc = []
    yc = []
    
    for i in range(len(X)):
        a = fitCircle(newX[i:i+2*step],newY[i:i+2*step])
        xc.append(a[0])
        yc.append(a[1])
        Curv.append(1/a[2])
        
    return(xc,yc,Curv)


#### Smoothing contour and getting landmarks 

In [ ]:
def getLandmarks(CD,GD,StackList,P,stringName, **kwargs): 
        
    # Utility function
    def round_to_odd(f):
        return np.floor(f) // 2 * 2 + 1    
    
    # Folder definition
    if not os.path.exists(P + '\\Figures\\'):
        os.mkdir(P + '\\Figures\\') # create folder
        
    if not os.path.exists(P + '\\Figures\\Landmarks\\'):
        os.mkdir(P + '\\Figures\\Landmarks\\') # create folder
        
    # Kwargs
    DebugPlots = False
    SavedPlots = False
    Dmax = 20
    AUTO = False
    OverwriteData = False
    
    for key, value in kwargs.items(): 
        if key == 'debug':
            DebugPlots = value
        elif key == 'saveplots':
            SavedPlots = value
        elif key == 'Dmax':
            Dmax = value
        elif key == 'Auto':
            AUTO = value
        elif key == 'Overwrite':
            OverwriteData = value             
        elif key == 'FirstSlice':
            FirstSlice = value
        else:
            print('Unknown key : ' + key + '. Kwarg ignored.')
        
    # Load or create file for landmark points 
    if os.path.exists(P + '\\clickedpoints.csv'):
        RefPts = pd.read_csv(P + '\\clickedpoints.csv', index_col = 'Ind')
    else:                
        RefPts = pd.DataFrame(data=None,columns=['Img','Xnotch1','Ynotch1','Xnotch2','Ynotch2','Xattach','Yattach']) 
    
    # 1. For each video in StackList identify landmarks on first image
    
    for s in StackList:
        
        if not os.path.exists(P + '\\Figures\\Landmarks\\'+ s + '\\'):
            os.mkdir(P + '\\Figures\\Landmarks\\'+ s + '\\') # create folder
            
        print('First image landmarks for : ' + s.ljust(10), flush=True, end = '\r')
        
        i = 0 # First image
        
        if ('Snotch2' in GD) & (not OverwriteData):
            if (np.isnan(GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Snotch2'].values[:])):
                DO = True
            else:
                DO = False
        else:
            DO = True
        
        if DO:
        
            # Loading contour and center coordinate
            Xcenter = GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Xcenter'].values
            Ycenter = GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Ycenter'].values

            Xcont = CD.loc[(CD.index == s) & (CD['Img'] == i) ,'Ximg'].values
            Ycont = CD.loc[(CD.index == s) & (CD['Img'] == i) ,'Yimg'].values

            # Contour smoothing
            window = int(round_to_odd(len(Xcont)/50))

            Xsmooth = savgol_filter(Xcont, window, 3)
            Ysmooth = savgol_filter(Ycont, window, 3)

            # Save smoothed contour
            CD.loc[(CD.index == s) & (CD['Img'] == i) ,'Xsmooth'] = Xsmooth
            CD.loc[(CD.index == s) & (CD['Img'] == i) ,'Ysmooth'] = Ysmooth

            ### Automaticaly detect points of high inward curvature  
            # Compute curvature
            Step = round(2/100*len(Xsmooth))
            xc,yc,Curv = getContourCurvature(Xsmooth,Ysmooth,Step)

            # Compute signed curvature using the position of the center of the fitted 
            # circle with regards to the contour
            poly = [(x,y) for (x,y) in zip(Xsmooth,Ysmooth)]
            points = [[x,y] for (x,y) in zip(xc,yc)]
            path = mpltPath.Path(poly)            
            isIn = path.contains_points(points)

            CurvSign = []
            for Bool in isIn:
                if Bool:
                    CurvSign.append(1)
                else:
                    CurvSign.append(-1)

            SignedCurv = np.multiply(Curv,CurvSign)

            # Detecting negative peaks in signed curvature
            loc,prop = find_peaks(-SignedCurv, distance = len(SignedCurv)/50) 


            ### Identifying landmarks
            # if clicked points not already in the file, ask user and then save in the file
            if RefPts.loc[(RefPts.index == s)&(RefPts['Img']==i)].empty:

                # identifying notches manually on fist image
                NotchesRef = getContourPointsCoordinates(P,s,2,i,Xsmooth,Ysmooth,Xsmooth[loc],Ysmooth[loc],Xsmooth,Ysmooth,Xsmooth[loc],Ysmooth[loc])
                AttachRef = getContourPointsCoordinates(P,s,1,i,Xsmooth,Ysmooth,Xsmooth[loc],Ysmooth[loc],Xsmooth,Ysmooth,Xsmooth[loc],Ysmooth[loc])

                # Save the points that were clicked
                data = {'Img': i,
                        'Xnotch1':NotchesRef[0][0],
                        'Ynotch1':NotchesRef[0][1],
                        'Xnotch2':NotchesRef[1][0],
                        'Ynotch2':NotchesRef[1][1],
                        'Xattach':AttachRef[0][0],
                        'Yattach':AttachRef[0][1]}            

                RefPts = RefPts.append(pd.DataFrame(data=data,index = [s]))

                RefPts.to_csv(P + '\\clickedpoints.csv',index_label = 'Ind')

            else: # Load clicked points from the file
                    NotchesRef = [[RefPts.loc[(RefPts.index == s) & (RefPts['Img'] == i) ,'Xnotch1'].values,
                                       RefPts.loc[(RefPts.index == s) & (RefPts['Img'] == i) ,'Ynotch1'].values],
                                     [RefPts.loc[(RefPts.index == s) & (RefPts['Img'] == i) ,'Xnotch2'].values,
                                       RefPts.loc[(RefPts.index == s) & (RefPts['Img'] == i) ,'Ynotch2'].values]]
                    AttachRef = [[RefPts.loc[(RefPts.index == s) & (RefPts['Img'] == i) ,'Xattach'].values, 
                                 RefPts.loc[(RefPts.index == s) & (RefPts['Img'] == i) ,'Yattach'].values]]


            # Find landmarks in contour from clicked points and Curvature computation   
            xN1 = Xsmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-NotchesRef[0][0])+
                                                  np.square(Ysmooth[loc]-NotchesRef[0][1])))]  
            yN1 = Ysmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-NotchesRef[0][0])+
                                                  np.square(Ysmooth[loc]-NotchesRef[0][1])))] 

            xN2 = Xsmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-NotchesRef[1][0])+
                                                  np.square(Ysmooth[loc]-NotchesRef[1][1])))]  
            yN2 = Ysmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-NotchesRef[1][0])+
                                                  np.square(Ysmooth[loc]-NotchesRef[1][1])))] 

            xA = Xsmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-AttachRef[0][0])+
                                                  np.square(Ysmooth[loc]-AttachRef[0][1])))]  
            yA = Ysmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-AttachRef[0][0])+
                                                  np.square(Ysmooth[loc]-AttachRef[0][1])))] 

            # Save landmarks points
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Xattach'] = xA
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Yattach'] = yA

            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Xnotch1'] = xN1
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Ynotch1'] = yN1

            GD.loc[(GD.index == s) & (GD['Img'] == i),'Xnotch2'] = xN2
            GD.loc[(GD.index == s) & (GD['Img'] == i),'Ynotch2'] = yN2


            # Find and save the positions in contour of notches points
            locn1 = np.argmin(np.abs(np.sqrt(np.square(Xsmooth-xN1)+np.square(Ysmooth-yN1))))
            locn2 = np.argmin(np.abs(np.sqrt(np.square(Xsmooth-xN2)+np.square(Ysmooth-yN2))))

            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'LocNotch1'] = locn1
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'LocNotch2'] = locn2

            # Plotting 
            if SavedPlots:
                RGBimg = io.imread(P + '\\' + s + '.tif', key = i)

                fig0, [ax0, ax1] = plt.subplots(ncols = 2, dpi = 250,facecolor='black')
                fig0.suptitle(s)
                ax0.set_title('Attach point (magenta), notches (red),\n and center (blue) for alignement',fontsize=8)
                ax0.imshow(RGBimg)
                ax0.plot(Xsmooth,Ysmooth,'c',lw = 0.5)
                ax0.plot(xN1,yN1,'.r',ms=2)
                ax0.plot(xN2,yN2,'.r',ms=2)
                ax0.plot(xA,yA,'.m',ms=2)
                ax0.plot(Xcenter[0],Ycenter[0],'*b', ms = 3)
                ax0.set_xticks([], [])
                ax0.set_yticks([], [])
                sc = ax1.scatter(Xsmooth,-Ysmooth+2*Ycenter[0], c= SignedCurv, cmap = 'BrBG', s = 2)
                ax1.plot(xN1,-yN1+2*Ycenter[0],'ob',ms=3)
                ax1.plot(xN2,-yN2+2*Ycenter[0],'ob',ms=3)
                ax1.plot(xA,-yA+2*Ycenter[0],'sb',ms=3)
                ax1.plot(Xsmooth[loc],-Ysmooth[loc]+2*Ycenter[0],'r+',mfc='none', ms = 3, lw= 0.5)
                ax1.set_aspect('equal')
                ax1.set_xticks([], [])
                ax1.set_yticks([], [])
                #ax1.set_xlim(left=-250+Xcenter[0],right=250+Xcenter[0])
                #ax1.set_ylim(top=250+Ycenter[0],bottom=-250+Ycenter[0])
                fig0.colorbar(sc, ax = ax1, label = 'Curvature',shrink = 0.6)
                fig0.tight_layout()

                fig0.savefig(P + '\\Figures\\Landmarks\\'+ s + '\\' + str(i) +'.png') 
                plt.close()

    # Save tmp file with already analyzed stacks        
    GD.to_csv(P + '\\GlobalData' + stringName + '_Landmarks_tmp.csv',index_label = 'Ind')
    CD.to_csv(P + '\\ContourData' + stringName + '_Landmarks_tmp.csv',index_label = 'Ind')
    
    print('\n')
    
    if not FirstSlice:
        # 2. For the remaining image, detect automatically landmark points
        for s in StackList:       

            nimg = int(1 + np.max(GD.loc[s, 'Img'])) # number of images in the stack

            if  OverwriteData:
                idx = 1
            else:            
                List = GD.loc[s,'Snotch2'].values
                idx = next((i for i in range(len(List)) if np.isnan(List[i])),nimg)

#             print('nimg : ' + str(nimg) + ' // idx : ' + str(idx))

            if idx == nimg:
                print('Stack ' + s + ' already processed in tmp file.', end='')
            else:
                for i in range(idx,nimg):

                    print('Processing ' + s + ' image ' + str(i+1) + '/' + str(nimg).ljust(10), flush=True, end = '\r')

                    # Loading contour and center coordinate
                    Xcenter = GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Xcenter'].values
                    Ycenter = GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Ycenter'].values

                    Xcont = CD.loc[(CD.index == s) & (CD['Img'] == i) ,'Ximg'].values
                    Ycont = CD.loc[(CD.index == s) & (CD['Img'] == i) ,'Yimg'].values

                    # Contour smoothing
                    window = int(round_to_odd(len(Xcont)/50))

                    Xsmooth = savgol_filter(Xcont, window, 3)
                    Ysmooth = savgol_filter(Ycont, window, 3)

                    # Save smoothed contour
                    CD.loc[(CD.index == s) & (CD['Img'] == i) ,'Xsmooth'] = Xsmooth
                    CD.loc[(CD.index == s) & (CD['Img'] == i) ,'Ysmooth'] = Ysmooth

                    # Center smoothed contour on (0,0)
                    Xcent = Xsmooth - Xcenter
                    Ycent = Ysmooth - Ycenter

                    ### Automaticaly detect points of high inward curvature  
                    # Compute curvature
                    Step = round(2/100*len(Xsmooth))
                    xc,yc,Curv = getContourCurvature(Xsmooth,Ysmooth,Step)

                    # Compute signed curvature using the position of the center of the fitted 
                    # circle with regards to the contour
                    poly = [(x,y) for (x,y) in zip(Xsmooth,Ysmooth)]
                    points = [[x,y] for (x,y) in zip(xc,yc)]
                    path = mpltPath.Path(poly)            
                    isIn = path.contains_points(points)

                    CurvSign = []
                    for Bool in isIn:
                        if Bool:
                            CurvSign.append(1)
                        else:
                            CurvSign.append(-1)

                    SignedCurv = np.multiply(Curv,CurvSign)

                    # Detecting peaks in signed curvature
                    loc,prop = find_peaks(-SignedCurv, distance = len(SignedCurv)/30)             

                    ### Identifying landmarks
                    # if there is no reference point in the file
                    if RefPts.loc[(RefPts.index == s)&(RefPts['Img']==i)].empty:

                        # Getting centered contour from previous image 
                        XsmoothOld = CD.loc[(CD.index == s) & (CD['Img'] == i-1) ,'Xsmooth'].values
                        YsmoothOld = CD.loc[(CD.index == s) & (CD['Img'] == i-1) ,'Ysmooth'].values

                        XcenterOld = GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Xcenter'].values
                        YcenterOld = GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Ycenter'].values

                        XcentOld = XsmoothOld-XcenterOld
                        YcentOld = YsmoothOld-YcenterOld

                        ### Rotate new contour to align on old one
                        # Expressed in polar coordinates
                        Theta,R = vf.ToCirc(XcentOld,YcentOld,angle = 'deg')
                        Rs = mtl.repmat(R,360,1)

                        # Rotation steps (1°)
                        rot = np.transpose(np.array([range(0,360)]))
                        Thetas = np.mod(Theta + rot,360)

                        Xmatrot, Ymatrot = vf.ToCart(Thetas,Rs, angle = 'deg')

                        Dvect = np.empty(360)

                        for d in range(360):
                            Nxy = len(Xcent)
                            minD = np.empty(Nxy)
                            for (x,y,nxy) in zip(Xcent,Ycent,range(Nxy)):
                                 minD[nxy] = np.min(vf.dist(x,y,Xmatrot[d,:],Ymatrot[d,:]))
                            Dvect[d] = sum(np.square(minD))

                        m = np.argmin(Dvect)

                        XalOld = Xmatrot[m,:]+Xcenter
                        YalOld = Ymatrot[m,:]+Ycenter

                        if DebugPlots:
                            mp = np.mod(m+5,359)
                            mm = np.mod(m-5,359)
                            fig,ax = plt.subplots(dpi=200)
                            ax.set_title('Contour alignement for image : ' +str(i) + ' m = ' + str(m) )
                            ax.set_aspect('equal')
                            ax.plot(XsmoothOld,YsmoothOld,'b--',label = ('i-1 contour'))
                            ax.plot(Xsmooth,Ysmooth,'w--', label = ('i contour'))
                            ax.plot(XalOld,YalOld,'g',label = ('translated and rotated i-1 contour'))
                            ax.plot(Xmatrot[mp,:]+Xcenter,Ymatrot[mp,:]+Ycenter,'r',lw=1,label = ('translated and rotated i-1 contour'))
                            ax.plot(Xmatrot[mm,:]+Xcenter,Ymatrot[mm,:]+Ycenter,'r--',lw=1,label = ('translated and rotated i-1 contour'))
                            plt.legend(fontsize = 'xx-small')
                            plt.show()



                        ### if high curvature points are too far from old 
                        ### aligned contour landmarks, ask again the user

                        # landmark points on old contour, centered
                        xN1Old = GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Xnotch1'].values-XcenterOld
                        yN1Old = GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Ynotch1'].values-YcenterOld
                        xN2Old = GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Xnotch2'].values-XcenterOld
                        yN2Old = GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Ynotch2'].values-YcenterOld
                        xAOld = GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Xattach'].values-XcenterOld
                        yAOld = GD.loc[(GD.index == s) & (GD['Img'] == i-1) ,'Yattach'].values-YcenterOld


                        # rotation of old landmark points
                        ThetaN1Old,RN1Old = vf.ToCirc(xN1Old,yN1Old, angle = 'deg')
                        xN1OldRot,yN1OldRot = vf.ToCart(ThetaN1Old+m,RN1Old, angle = 'deg')
                        ThetaN2Old,RN2Old = vf.ToCirc(xN2Old,yN2Old, angle = 'deg')
                        xN2OldRot,yN2OldRot = vf.ToCart(ThetaN2Old+m,RN2Old, angle = 'deg')
                        ThetaAOld,RAOld = vf.ToCirc(xAOld,yAOld, angle = 'deg')
                        xAOldRot,yAOldRot = vf.ToCart(ThetaAOld+m,RAOld, angle = 'deg')

                        # Old landmark points aligned to new contour
                        xN1OldAl = xN1OldRot+Xcenter
                        yN1OldAl = yN1OldRot+Ycenter
                        xN2OldAl = xN2OldRot+Xcenter
                        yN2OldAl = yN2OldRot+Ycenter
                        xAOldAl = xAOldRot+Xcenter
                        yAOldAl = yAOldRot+Ycenter


                        DN1 = np.min(vf.dist(xN1OldAl,yN1OldAl,Xsmooth[loc],Ysmooth[loc]))
                        DN2 = np.min(vf.dist(xN2OldAl,yN2OldAl,Xsmooth[loc],Ysmooth[loc]))
                        DN = np.min([DN1,DN2])

                        DA = np.min(vf.dist(xAOldAl,yAOldAl,Xsmooth[loc],Ysmooth[loc]))

                        if (DN > Dmax) & (vf.HausdorffDist(XalOld,YalOld,Xsmooth,Ysmooth)>Dmax):
                            if AUTO:
                                print('Processing ' + s + ' image ' + str(i+1) + '/' + str(nimg).ljust(10), flush=True, end = '\n')
                                print('User input needed, skiping this in Automode !')
                                break
                            else:
                                NotchesRef = getContourPointsCoordinates(P,s,2,i,Xsmooth,Ysmooth,Xsmooth[loc],Ysmooth[loc],XalOld,YalOld,[xN1OldAl,xN2OldAl],[yN1OldAl,yN2OldAl])

                        else:
                            # aligned old landlark points (= new reference points)
                            NotchesRef = [[xN1OldAl,yN1OldAl],
                                         [xN2OldAl,yN2OldAl]]

                        if (DA > Dmax) & (vf.HausdorffDist(XalOld,YalOld,Xsmooth,Ysmooth)>Dmax):

                            if AUTO:
                                print('Processing ' + s + ' image ' + str(i+1) + '/' + str(nimg).ljust(10), flush=True, end = '\n')
                                print('User input needed, skiping this in Automode !')
                                break
                            else:
                                AttachRef = getContourPointsCoordinates(P,s,1,i,Xsmooth,Ysmooth,Xsmooth[loc],Ysmooth[loc],XalOld,YalOld,xAOldAl,yAOldAl)

                        else:  
                            # aligned old landlark points (= new reference points)
                            AttachRef = [[xAOldAl,yAOldAl]]



                        # Save the reference points
                        data = {'Img': i,
                                'Xnotch1':NotchesRef[0][0],
                                'Ynotch1':NotchesRef[0][1],
                                'Xnotch2':NotchesRef[1][0],
                                'Ynotch2':NotchesRef[1][1],
                                'Xattach':AttachRef[0][0],
                                'Yattach':AttachRef[0][1]}            

                        RefPts = RefPts.append(pd.DataFrame(data=data,index = [s]))

                        RefPts.to_csv(P + '\\clickedpoints.csv',index_label = 'Ind')
                        
                    else:
                        NotchesRef = [[RefPts.loc[(RefPts.index == s) & (RefPts['Img'] == i) ,'Xnotch1'].values,
                                           RefPts.loc[(RefPts.index == s) & (RefPts['Img'] == i) ,'Ynotch1'].values],
                                         [RefPts.loc[(RefPts.index == s) & (RefPts['Img'] == i) ,'Xnotch2'].values,
                                           RefPts.loc[(RefPts.index == s) & (RefPts['Img'] == i) ,'Ynotch2'].values]]
                        AttachRef = [[RefPts.loc[(RefPts.index == s) & (RefPts['Img'] == i) ,'Xattach'].values, 
                                     RefPts.loc[(RefPts.index == s) & (RefPts['Img'] == i) ,'Yattach'].values]]


                    # Saving landmarks    
                    xN1 = Xsmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-NotchesRef[0][0])+
                                                          np.square(Ysmooth[loc]-NotchesRef[0][1])))]  
                    yN1 = Ysmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-NotchesRef[0][0])+
                                                          np.square(Ysmooth[loc]-NotchesRef[0][1])))] 

                    xN2 = Xsmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-NotchesRef[1][0])+
                                                          np.square(Ysmooth[loc]-NotchesRef[1][1])))]  
                    yN2 = Ysmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-NotchesRef[1][0])+
                                                          np.square(Ysmooth[loc]-NotchesRef[1][1])))] 

                    xA = Xsmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-AttachRef[0][0])+
                                                          np.square(Ysmooth[loc]-AttachRef[0][1])))]  
                    yA = Ysmooth[loc][np.argmin(np.sqrt(np.square(Xsmooth[loc]-AttachRef[0][0])+
                                                          np.square(Ysmooth[loc]-AttachRef[0][1])))] 

                    GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Xattach'] = xA
                    GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Yattach'] = yA

                    GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Xnotch1'] = xN1
                    GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Ynotch1'] = yN1

                    GD.loc[(GD.index == s) & (GD['Img'] == i),'Xnotch2'] = xN2
                    GD.loc[(GD.index == s) & (GD['Img'] == i),'Ynotch2'] = yN2

                    
                    # Find and save position of notches 
                    locn1 = np.argmin(np.abs(np.sqrt(np.square(Xsmooth-xN2)+np.square(Ysmooth-yN2))))
                    locn2 = np.argmin(np.abs(np.sqrt(np.square(Xsmooth-xN1)+np.square(Ysmooth-yN1))))
                    
                    GD.loc[(GD.index == s) & (GD['Img'] == i) ,'LocNotch1'] = locn1
                    GD.loc[(GD.index == s) & (GD['Img'] == i) ,'LocNotch2'] = locn2

                    # plotting
                    if SavedPlots:
                        RGBimg = io.imread(P + '\\' + s + '.tif', key = i)

                        fig0, [ax0, ax1] = plt.subplots(ncols = 2, dpi = 250,facecolor='black')
                        fig0.suptitle(s)
                        ax0.set_title('Attach point (magenta), notches (red),\n and center (blue) for alignement',fontsize=8)
                        ax0.imshow(RGBimg)
                        ax0.plot(Xsmooth,Ysmooth,'c',lw = 0.5)
                        ax0.plot(xN1,yN1,'.r',ms=2)
                        ax0.plot(xN2,yN2,'.r',ms=2)
                        ax0.plot(xA,yA,'.m',ms=2)
                        ax0.plot(Xcenter[0],Ycenter[0],'*b', ms = 3)
                        ax0.set_xticks([], [])
                        ax0.set_yticks([], [])
                        sc = ax1.scatter(Xsmooth,-Ysmooth+2*Ycenter[0], c= SignedCurv, cmap = 'BrBG', s = 2)
                        ax1.plot(xN1,-yN1+2*Ycenter[0],'ob',ms=3)
                        ax1.plot(xN2,-yN2+2*Ycenter[0],'ob',ms=3)
                        ax1.plot(xA,-yA+2*Ycenter[0],'sb',ms=3)
                        ax1.plot(Xsmooth[loc],-Ysmooth[loc]+2*Ycenter[0],'r+',mfc='none', ms = 3, lw= 0.5)
                        ax1.set_aspect('equal')
                        ax1.set_xticks([], [])
                        ax1.set_yticks([], [])
                        #ax1.set_xlim(left=-250+Xcenter[0],right=250+Xcenter[0])
                        #ax1.set_ylim(top=250+Ycenter[0],bottom=-250+Ycenter[0])
                        fig0.colorbar(sc, ax = ax1, label = 'Curvature',shrink = 0.6)
                        fig0.tight_layout()

                        fig0.savefig(P + '\\Figures\\Landmarks\\'+ s + '\\' + str(i) +'.png') 
                        plt.close()

                # Save tmp file with already analyzed stacks        
                GD.to_csv(P + '\\GlobalData' + stringName + '_Landmarks_tmp.csv',index_label = 'Ind')
                CD.to_csv(P + '\\ContourData' + stringName + '_Landmarks_tmp.csv',index_label = 'Ind')

            print('\n')

    return(CD,GD)

### Defining curvilinear-like abscisse with landmarks

In [ ]:
def curvAbsci(CD,GD,StackList,P, **kwargs):

    if not os.path.exists(P + '\\Figures\\Parametrisation\\'):
        os.mkdir(P + '\\Figures\\Parametrisation\\') # create folder
    
    DebugPlots = False
    SavedPlots = False
    
    for key, value in kwargs.items(): 
        if key == 'debug':
            DebugPlots = value
        if key == 'saveplots':
            SavedPlots = value
        else:
            print('Unknown key : ' + key + '. Kwarg ignored.')
    
    newCD = pd.DataFrame(data=None,columns=['Img','Xparam','Yparam','Sparam']) 
    
    
    for s in StackList:
        
        print('Processing ' + s + '...',end=' ')
        
        if not os.path.exists(P + '\\Figures\\Parametrisation\\'+ s + '\\'):
            os.mkdir(P + '\\Figures\\Parametrisation\\'+ s + '\\') # create folder
            
        n = int(1 + np.max(CD.loc[s, 'Img']))
        
        for i in range(n): # 
            
            # retrieve important points and contours
            Xcont = CD.loc[(CD.index == s) & (CD['Img'] == i), 'Xsmooth'].values
            Ycont = CD.loc[(CD.index == s) & (CD['Img'] == i), 'Ysmooth'].values            
            
            Xc = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xcenter'].values
            Yc = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Ycenter'].values
            
            Xat = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xattach'].values
            Yat = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Yattach'].values

            Xn1 = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xnotch1'].values
            Yn1 = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Ynotch1'].values

            Xn2 = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xnotch2'].values
            Yn2 = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Ynotch2'].values
            
            locn1 = np.argmin(np.abs(np.sqrt(np.square(Xcont-Xn1)+np.square(Ycont-Yn1))))
            locn2 = np.argmin(np.abs(np.sqrt(np.square(Xcont-Xn2)+np.square(Ycont-Yn2))))
            
            
            # identify 'first notch' as the one being after the attach point clockwise
            # Shift the contour points so that they start by the first notch
            Aat,Rat = vf.ToCirc(Xat-Xc,Yat-Yc,angle='deg')
            An1,Rn1 = vf.ToCirc(Xn1-Xc,Yn1-Yc,angle='deg')
            An2,Rn2 = vf.ToCirc(Xn2-Xc,Yn2-Yc,angle='deg')
            
            An1=np.mod(An1-Aat,360)
            An2=np.mod(An2-Aat,360)
            
            if An1>An2: # n1 is first notch
                Xcont = np.roll(Xcont,-locn1)
                Ycont = np.roll(Ycont,-locn1)
                locn = locn2-locn1 # non starting notch
                GD.loc[(GD.index == s) & (GD['Img'] == i) ,'LocNotch1Param'] = 0
                GD.loc[(GD.index == s) & (GD['Img'] == i),'LocNotch2Param'] = locn
                    
            else: # n2 is first notch
                Xcont = np.roll(Xcont,-locn2)
                Ycont = np.roll(Ycont,-locn2)
                locn = locn1-locn2 # non starting notch
                GD.loc[(GD.index == s) & (GD['Img'] == i) ,'LocNotch1Param'] = locn
                GD.loc[(GD.index == s) & (GD['Img'] == i),'LocNotch2Param'] = 0
            

            # Computes length along contour and total length
            ContourCumLength = np.concatenate(([0],np.cumsum(np.sqrt(np.square(np.diff(Xcont))+np.square(np.diff(Ycont))))))
            ContourLength = ContourCumLength[-1]
            
            # Create contour interpolation function based on length along contour
            ContourInterp = interp1d(ContourCumLength,[Xcont,Ycont], fill_value='extrapolate')            
            
            ## Create parametrisation for two section of contour : upper section between the two notches not containing the attach 
            ## point & lower section between the two noteches containing the attach points
            
            npts = 500 # number of point per section of contour
            
            # Upper section
            SegmentLength_up = ContourCumLength[locn]
            deltaL_up = SegmentLength_up/npts
            SegmentRegCumLength_up = np.linspace(0,npts,npts+1)*deltaL_up

            Xparam_up,Yparam_up = ContourInterp(SegmentRegCumLength_up[0:-1])
            Sparam_up = (SegmentRegCumLength_up[0:-1])/ContourLength
            
            CurvContourLength_up = np.sum(np.sqrt(np.square(np.diff(Xparam_up))+np.square(np.diff(Yparam_up))))
            
            # Lower section
            SegmentLength_lo = ContourLength-ContourCumLength[locn]
            deltaL_lo = SegmentLength_lo/npts 
            SegmentRegCumLength_lo = np.linspace(0,npts,npts+1)*deltaL_lo+SegmentLength_up

            Xparam_lo,Yparam_lo = ContourInterp(SegmentRegCumLength_lo[0:-1])
            Sparam_lo = (SegmentRegCumLength_lo[0:-1])/ContourLength
            
            CurvContourLength_lo = np.sum(np.sqrt(np.square(np.diff(Xparam_lo))+np.square(np.diff(Yparam_lo))))
            
            
            if DebugPlots:
                print(s + '_' + str(i) + ' contour length variation : ' + 
                      str(round((ContourLength-(CurvContourLength_up+CurvContourLength_lo+deltaL_up+deltaL_lo))
                                /ContourLength*10000)/100) + '%')

                
            Xparam = np.concatenate((Xparam_up, Xparam_lo))
            Yparam = np.concatenate((Yparam_up, Yparam_lo))
            Sparam = np.concatenate((Sparam_up, Sparam_lo))
            
            if SavedPlots|DebugPlots:
                
                loca = np.argmin(np.abs(np.sqrt(np.square(Xparam-Xat)+np.square(Yparam-Yat))))
                
                fig1, ax1 = plt.subplots(dpi = 200,facecolor='black')
                fig1.suptitle(s)
                ax1.set_title("Regular parametrisation by segment (1/10 pts)")
                ax1.plot(Xparam-Xc,Yparam-Yc,'w-')
                ax1.plot(0,0,'w*')
                ax1.plot(Xparam[0]-Xc,Yparam[0]-Yc,'mo',ms = 5, label='First notch')
                ax1.plot(Xcont[locn]-Xc,Ycont[locn]-Yc,'co',ms = 5, label='Second notch')
                ax1.plot(Xparam[loca]-Xc,Yparam[loca]-Yc,'go',ms = 5, label='Attach')
                ax1.plot(Xparam_up[0:len(Xparam_up)-1:10]-Xc,Yparam_up[0:len(Xparam_up)-1:10]-Yc,'bo',ms = 3,label='Upper section')
                ax1.plot(Xparam_lo[0:len(Xparam_lo)-1:10]-Xc,Yparam_lo[0:len(Xparam_lo)-1:10]-Yc,'ro',ms = 3, label='Lower section')
                ax1.set_aspect('equal', adjustable='box')
                plt.legend(fontsize = 'xx-small')
                fig1.savefig(P + '\\Figures\\Parametrisation\\'+ s + '\\' + 'Reg_' + str(i) +'.png') 
                if DebugPlots & ((i == 0)|(i == 20)|(i == 30)|(i == 10)):
                    plt.show()
                else:
                    plt.close()
                
            # Contour normalisation
            Thp,Rp = vf.ToCirc(Xparam,Yparam)
            XcN,YcN = vf.ToCart(Thp,np.divide(Rp,np.median(Rp)))
            
            # Storing contour data
            data = {'Img': i*np.ones(len(Xparam)),
                        'Xparam': Xparam,
                        'Yparam': Yparam,
                        'XparamNorm': XcN,
                        'YparamNorm': YcN,
                        'Sparam': Sparam} 
            

            
            newCD = newCD.append(pd.DataFrame(data=data,index = np.repeat(s,len(Xparam))) )            
        print('Done')
            
    return(newCD,GD)

### Reorienting contours and landmarks

Reorientation is done by first centering the contour on the origin, then rotating until alignement of the bissector of the notches-center axes with the horizontal.

In [ ]:
def rotateAndCenterShape(CD,GD,StackList,P,Scale, **kwargs):
    
    if not os.path.exists(P + '\\Figures\\Rotation\\'):
        os.mkdir(P + '\\Figures\\Rotation\\') # create folder
    
    DebugPlots = False
    SavedPlots = False
    
    for key, value in kwargs.items(): 
        if key == 'debug':
            DebugPlots = value
        elif key == 'saveplots':
            SavedPlots = value        
        else:
            print('Unknown key : ' + key + '. Kwarg ignored.')
    
    newCD = pd.DataFrame(data=None,columns=['Img','Ximg','Yimg','XAligned','YAligned','Xsmooth','Ysmooth','InitialS']) 
    
    for s in StackList:
        
        
        if not os.path.exists(P + '\\Figures\\Rotation\\'+ s + '\\'):
            os.mkdir(P + '\\Figures\\Rotation\\'+ s + '\\') # create folder
        
        n = int(1 + np.max(CD.loc[s, 'Img']))
        
        for i in range(n):
            # retrieve important points and contours
            Xc = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xcenter'].values
            Yc = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Ycenter'].values

            Xcont = CD.loc[(CD.index == s) & (CD['Img'] == i), 'Xsmooth'].values-Xc
            Ycont = CD.loc[(CD.index == s) & (CD['Img'] == i), 'Ysmooth'].values-Yc

            XA = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xattach'].values-Xc
            YA = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Yattach'].values-Yc

            Xn1 = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xnotch1'].values-Xc
            Yn1 = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Ynotch1'].values-Yc

            Xn2 = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xnotch2'].values-Xc
            Yn2 = GD.loc[(GD.index == s) & (GD['Img'] == i), 'Ynotch2'].values-Yc


            A,R = vf.ToCirc([XA,Xn1,Xn2],[YA,Yn1,Yn2],angle='deg')
            Theta,Radius = vf.ToCirc(Xcont,Ycont,angle='deg')
            
            Anotches = np.mod(A[1:3].T-np.array([0,180]),360)
            
            Abis = np.mod(Anotches.sum()/2,360)-180
            
            [XAal,Xn1al,Xn2al],[YAal,Yn1al,Yn2al] = vf.ToCart(A-Abis,R,angle='deg')
            
            if YAal > 0:
                Abis = Abis + 180
                [XAal,Xn1al,Xn2al],[YAal,Yn1al,Yn2al] = vf.ToCart(A-Abis,R,angle='deg')

            Xcontal,Ycontal = vf.ToCart(Theta-Abis,Radius,angle='deg')

            # Start abcisse from attach points
            locA = np.argmin(np.abs(np.sqrt(np.square(Xcontal-XAal)+np.square(Ycontal-YAal))))
            
            Xcontal = np.roll(Xcontal,-locA)
            Ycontal = np.roll(Ycontal,-locA)

            # Storing contour data
            data = {'Img': i*np.ones(len(Xcontal)), 
                        'Ximg': Xcont+Xc,  
                        'Yimg': Ycont+Yc,
                        'Xsmooth': CD.loc[(CD.index == s) & (CD['Img'] == i), 'Xsmooth'].values,  
                        'Ysmooth': CD.loc[(CD.index == s) & (CD['Img'] == i), 'Ysmooth'].values, 
                        'InitialS': CD.loc[(CD.index == s) & (CD['Img'] == i), 'InitialS'].values,
                        'Xcentered': Xcont,
                        'Ycentered': Ycont,
                        'XAligned': Xcontal*Scale,
                        'YAligned': Ycontal*Scale} 
            
            newCD = newCD.append(pd.DataFrame(data=data,index = np.repeat(s,len(Xcont))) )
            
            # create aligned parametrisation
            npts = len(Xcontal)
            
            S = np.linspace(0,1,npts)
            
            locn1al = np.argmin(np.abs(np.sqrt(np.square(Xcontal-Xn2al)+np.square(Ycontal-Yn2al))))
            locn2al = np.argmin(np.abs(np.sqrt(np.square(Xcontal-Xn1al)+np.square(Ycontal-Yn1al))))
            
            Sn1al = S[locn1al]
            Sn2al = S[locn2al]
            
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'LocNotch1Aligned'] = locn1al
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'LocNotch2Aligned'] = locn2al
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Snotch1Aligned'] = Sn1al
            GD.loc[(GD.index == s) & (GD['Img'] == i) ,'Snotch2Aligned'] = Sn2al
            
            GD.loc[(GD.index == s) & (GD['Img'] == i), 'XattachAligned'] = XAal*Scale
            GD.loc[(GD.index == s) & (GD['Img'] == i), 'YattachAligned'] = YAal*Scale

            GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xnotch1Aligned'] = Xn1al*Scale
            GD.loc[(GD.index == s) & (GD['Img'] == i), 'Ynotch1Aligned'] = Yn1al*Scale

            GD.loc[(GD.index == s) & (GD['Img'] == i), 'Xnotch2Aligned'] = Xn2al*Scale
            GD.loc[(GD.index == s) & (GD['Img'] == i), 'Ynotch2Aligned'] = Yn2al*Scale
        
            if SavedPlots|DebugPlots:
                fig,[ax0,ax1] = plt.subplots(ncols=2,dpi=200,facecolor='black')
                fig.suptitle(s)
                ax0.plot(Xcont,Ycont,'c')
                ax0.plot(XA,YA,'wo')
                ax0.plot(Xn1,Yn1,'r*',ms=5)
                ax0.plot(Xn2,Yn2,'m*',ms=5)
                ax0.plot(0,0,'b*')
                ax0.plot([0, Xn1],[0, Yn1],'m--')
                ax0.plot([0, Xn2],[0, Yn2],'r--')
                ax0.plot([0, Xn2],[0, Yn2],'r--')
                #ax0.set_xlim(left=-250,right=250)
                #ax0.set_ylim(top=250,bottom=-250)
                ax0.set_aspect('equal', adjustable='box')

                ax1.plot([-250,250],[0, 0],'w--',lw = 0.5)
                ax1.plot(Xcontal,Ycontal,'c')
                ax1.plot(XAal,YAal,'wo')
                ax1.plot(Xn1al,Yn1al,'r*',ms=5)
                ax1.plot(Xn2al,Yn2al,'m*',ms=5)
                ax1.plot(0,0,'b*')
                ax1.plot([0, Xn1al],[0, Yn1al],'m--')
                ax1.plot([0, Xn2al],[0, Yn2al],'r--')
                #ax1.set_xlim(left=-250,right=250)
                #ax1.set_ylim(top=250,bottom=-250)
                ax1.set_aspect('equal', adjustable='box')

                fig.tight_layout()
                fig.savefig(P + '\\Figures\\Rotation\\'+ s + '\\' + str(i) +'.png') 
                if DebugPlots & (i == 0):
                    plt.show()
                else:
                    plt.close()
        
    return(newCD,GD)

### Wrapper function for parametric contour computation

In [ ]:
def ParametriseContour(stringName,Path,dateCond,StackList,Scale,Todo, **kwargs):
    
    doL   = False
    doLPR = False
    doPR  = False
    doR   = False
    
    if Todo == 'LPR':        
        doLPR = True
    elif Todo == 'PR':
        doPR  = True
    elif Todo == 'R':
        doR   = True
    elif Todo == 'L':
        doL   = True
        
    DebugPlots = False  
    LdmkPlots = True
    Dmax = 20   
    AUTO = False
    OverwriteData = False
    FirstSlice = False
    
    for key, value in kwargs.items(): 
        if key == 'debug':
            DebugPlots = value
        elif key == 'ldmkplots':
            SavedPlots = value
        elif key == 'Dmax':
            Dmax = value
        elif key == 'AutoLdmks':
            AUTO = value
        elif key == 'Overwrite':
            OverwriteData = value
        elif key == 'FirstSlice':
            FirstSlice = value
        else:
            print('Unknown key : ' + key + '. Kwarg ignored.')
    
    print('\033[1m' + '\033[4m' + '\nAnalyzing ' + dateCond + ':\n' + '\033[0m')
    
    if doLPR|doL:
        ### Loading area and contour data
        if not os.path.exists(Path + '\\GlobalData' + stringName + '_Landmarks.csv'):
            if not os.path.exists(Path + '\\GlobalData' + stringName + '_Landmarks_tmp.csv'):
                ContourData = pd.read_csv(Path + '\\ContourData' + stringName + '_AreaCont.csv', index_col = 'Ind')
                GlobalData = pd.read_csv(Path + '\\GlobalData' + stringName + '_AreaCont.csv', index_col = 'Ind')
                print('\n Loaded AreaCont file.')
            else:            
                ContourData = pd.read_csv(Path + '\\ContourData' + stringName + '_Landmarks_tmp.csv', index_col = 'Ind')
                GlobalData = pd.read_csv(Path + '\\GlobalData' + stringName + '_Landmarks_tmp.csv', index_col = 'Ind')
                print('\n Loaded Landmarks_tmp file.')
        else:            
            ContourData = pd.read_csv(Path + '\\ContourData' + stringName + '_Landmarks.csv', index_col = 'Ind')
            GlobalData = pd.read_csv(Path + '\\GlobalData' + stringName + '_Landmarks.csv', index_col = 'Ind')
            print('\n Loaded Landmarks file.')

        print('\n\n\nGetting landmarks for : ' + dateCond + '\n\n')
        ContourData_LM, GlobalData_LM = getLandmarks(ContourData,GlobalData,StackList,Path,stringName, 
                                                     debug = DebugPlots, saveplots = LdmkPlots,
                                                     Dmax = Dmax, Auto = AUTO, Overwrite = OverwriteData,
                                                     FirstSlice = FirstSlice)
        
        if doLPR:
            # deleting tmp files
            os.remove(Path + '\\GlobalData' + stringName + '_Landmarks_tmp.csv')
            os.remove(Path + '\\ContourData' + stringName + '_Landmarks_tmp.csv')
            GlobalData_LM.to_csv(Path + '\\GlobalData' + stringName + '_Landmarks.csv',index_label = 'Ind')
            ContourData_LM.to_csv(Path + '\\ContourData' + stringName + '_Landmarks.csv',index_label = 'Ind')
            print('\nLandmarks saved.\n\n')

    elif doPR|doR:
        ### Loading landmarks
        GlobalData_LM = pd.read_csv(Path + '\\GlobalData' + stringName + '_Landmarks.csv',index_col = 'Ind')
        ContourData_LM = pd.read_csv(Path + '\\ContourData' + stringName + '_Landmarks.csv',index_col = 'Ind')

    if doLPR|doPR:
        print('\n\n\nComputing parametric contours for : ' + dateCond + '\n\n')
        ContourData_Param,GlobalData_Param = curvAbsci(ContourData_LM,GlobalData_LM,StackList,Path, debug = DebugPlots)
        ContourData_Param.to_csv(Path + '\\ContourData' + stringName + '_Param.csv',index_label = 'Ind')
        GlobalData_Param.to_csv(Path + '\\GlobalData' + stringName + '_Param.csv',index_label = 'Ind')
        print('\nParametric contours saved.\n\n')

    elif doR:
        ### loading non parametric contours
        GlobalData_RC = pd.read_csv(Path + '\\GlobalData' + stringName + '_Param.csv',index_col = 'Ind')
        ContourData_RC = pd.read_csv(Path + '\\ContourData' + stringName + '_Param.csv',index_col = 'Ind')

    if doLPR|doPR|doR:       
        print('\n\n\nAligning contours for : ' + dateCond + '\n\n')
        ContourData_RC,GlobalData_RC = rotateAndCenterShape(ContourData_Param,GlobalData_Param,StackList,Path,Scale, debug = DebugPlots)
        GlobalData_RC.to_csv(Path + '\\GlobalData' + stringName + '_ParamAligned.csv',index_label = 'Ind')
        ContourData_RC.to_csv(Path + '\\ContourData' + stringName + '_ParamAligned.csv',index_label = 'Ind')
        print('\nAligned parametric contours saved.\n\n')
        
    elif doL:
        print('Landmarks tmp round done for : ' + dateCond)
    else:
        print('No analysis done for : ' + dateCond)
        
    return

##  Enter data and run

### Data details

In [ ]:
Scale210903 = 1.94 # Spatial scale (µm/px) for 16X 03-09-2021
FPH210903 = 2 # Frames per hour

Scale210914 = 1.24 # Spatial scale (µm/px) for 25X 14-09-2021
FPH210914 = 2 # Frames per hour

Scale210927 = 1.24 # Spatial scale (µm/px) for 25X 27-09-2021
FPH210927 = 2 # Frames per hour

Scale211022 = 0.97 # Spatial scale (µm/px) for 32X 27-09-2021
FPH211022 = 2 # Frames per hour

Scale211105 = 1.55 # Spatial scale (µm/px) for 20X 05-11-2021
FPH211105 = 2 # Frames per hour

Scale211222 = 1.55 # Spatial scale (µm/px) for 20X 22-12-2021
FPH211222 = 2 # Frames per hour

Scale220107 = 1.94 # Spatial scale (µm/px) for 16X 07-01-2022
FPH220107 = 2 # Frames per hour

Scale220112 = 1.94 # Spatial scale (µm/px) for 16X 12-01-2022
FPH220112 = 2 # Frames per hour

Scale220114 = 1.94 # Spatial scale (µm/px) for 16X 14-01-2022
FPH220114 = 2 # Frames per hour

Scale220124 = 1.94 # Spatial scale (µm/px) for 16X 22-01-2022
FPH220124 = 2 # Frames per hour

# Paths to data
P210903_1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210903_V5_TestFlux\1mlh'
P210903_5 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210903_V5_TestFlux\500ulh'
P210914_Ct = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210914_V5_Manitol125mM\Ctrl'
P210914_M125 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210914_V5_Manitol125mM\Manitol125'
P210927_1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210927_V5_TestFlux\1mlh'
P210927_5 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\210927_V5_TestFlux\500ulh'
P211022_Ct1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211022_DV6_Ctrls\Ctrl1'
P211022_Ct2 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211022_DV6_Ctrls\Ctrl2'
P211105_Deg_Bsa = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211105_DV6_Degas_Ctrls\BSA'
P211105_Degas = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211105_DV6_Degas_Ctrls\NoCoat'
P211222_Deg_Bsa = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211222_DV6_Degas_Ctrls\BSA'
P211222_Degas = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\211222_DV6_Degas_Ctrls\NoCoat'
P220107_S3 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220107_DV6_Stade1&3\Stade3'
P220107_S1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220107_DV6_Stade1&3\Stade1'
P220112_S3 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220112_DV6_Stade1&3\Stade3'
P220112_S1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220112_DV6_Stade1&3\Stade1'
P220114_S3 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220114_DV6_Stade1&3\Stade3'
P220114_S1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220114_DV6_Stade1&3\Stade1'
P220124_Ct1 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220124_DV6_Ctrls\Ctrl1'
P220124_Ct2 = r'D:\Users\laplaud\Desktop\PostDoc\Data\Microflu\220124_DV6_Ctrls\Ctrl2'


# experiments of 03-09-2021, Puce V5, Ctrl with 1ml/heure medium flow
StackList210903_1 = ['PPG1','PPG2','PPG3','PPG6','PPG7','PPG10','PPG11','PPG12','PPG13','PPG14','PPG17','PPG19',
                   'PPG21','PPG22'] # 'PPG4','PPG5','PPG8','PPG9','PPG15','PPG16','PPG18','PPG20'

# experiments of 03-09-2021, Puce V5, Ctrl with 500ul/heure medium flow
StackList210903_5 = ['PPG3','PPG5','PPG6','PPG7','PPG8','PPG9','PPG10','PPG11'] # 'PPG1','PPG2','PPG4'

# experiments of 14-09-2021, Puce V5, Ctrl with 1ml/heure medium flow
StackList210914_Ct = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG9','PPG10','PPG11'] # 'PPG8','PPG12'

# experiments of 14-09-2021, Puce V5, 125mM manitol with 1ml/heure medium flow
StackList210914_M125 = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG7','PPG8','PPG9'] # 'PPG6'


# experiments of 27-09-2021, Puce V5, Ctrl with 1ml/heure medium flow
StackList210927_1 = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG7','PPG10',
                   'PPG11','PPG12','PPG13','PPG14','PPG15',] # 'PPG6','PPG8','PPG9'

# experiments of 27-09-2021, Puce V5, Ctrl with 500ul/heure medium flow
StackList210927_5 = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG8','PPG9','PPG10','PPG11','PPG13'] #'PPG7','PPG12'


# experiments of 22-10-2021, double puce V6, Ctrl 1 with 500ul/heure medium flow
StackList211022_Ct1 = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG8','PPG9',
                   'PPG11','PPG12','PPG13','PPG14','PPG15','PPG16','PPG17','PPG18','PPG19',
                       'PPG20','PPG21','PPG22','PPG23','PPG24','PPG25'] 
                    # 'PPG10', 
    
# experiments of 22-10-2021, double puce V6, Ctrl 2 with 500ul/heure medium flow
StackList211022_Ct2 = ['PPG1','PPG2','PPG4','PPG5','PPG6','PPG7','PPG8','PPG9','PPG10',
                   'PPG11','PPG12','PPG13','PPG15','PPG16','PPG18','PPG19','PPG20',
                   'PPG22','PPG23','PPG24','PPG26','PPG27','PPG28','PPG29']
                    # 'PPG3',  'PPG14', 'PPG17', 'PPG21', 'PPG25',


# experiments of 05-11-2021, double puce V6, Degased medium, BSA coated channel with 500ul/heure medium flow
StackList211105_Deg_Bsa = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG9','PPG10',
                     'PPG11', 'PPG13','PPG16','PPG18','PPG19','PPG20',
                   'PPG21','PPG23','PPG24','PPG25','PPG26','PPG27','PPG28','PPG29','PPG30']  #  
                       # PPG12, PPG8 dead , PPG15, PPG17 dead notch, PPG22 dead parts, PPG14 folded lost attach

# experiments of 05-11-2021, double puce V6, Degased medium  with 500ul/heure medium flow
StackList211105_Degas = ['PPG1','PPG2','PPG3','PPG4','PPG6','PPG7','PPG8','PPG9','PPG10',
                   'PPG11','PPG12','PPG13','PPG14','PPG16','PPG17','PPG18','PPG19','PPG20'] # 
                # PPG3 dead attach, PPG5 dead notch, 'PPG15' dead attach

    
# experiments of 22-12-2021, double puce V6, degased medium, bsa coating with 500ul/heure medium flow
# /!\ /!\ Start 30 minutes later than usual, first image removed
StackList211222_Deg_Bsa = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG8','PPG9','PPG10',
                   'PPG11','PPG12','PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20',
                   'PPG21','PPG22','PPG23','PPG24','PPG25','PPG26','PPG28','PPG29','PPG30',
                          'PPG31'] # PPG22 ,'PPG13' touche le bord de l'image # ,'PPG27' bad contouring


# experiments of 22-12-2021, double puce V6, degased medium, no coating with 500ul/heure medium flow
# /!\ /!\ Start 30 minutes later than usual, first image removed
StackList211222_Degas = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG9','PPG10',
                   'PPG11','PPG12','PPG13','PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20',
                   'PPG21','PPG22','PPG23','PPG24','PPG25','PPG26','PPG27','PPG28','PPG29','PPG30']
                    # PPG8 trop plié


# experiments of 07-01-2022, double puce V6, degased medium, no coating with 500ul/heure medium flow
# Stade1 (close to the center) cups
StackList220107_S1 = ['PPG1','PPG3','PPG6','PPG9',
                   'PPG11','PPG12','PPG13','PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20',
                   'PPG21','PPG23','PPG24','PPG25','PPG27']
                    # ,'PPG2','PPG7' contact avec autre PPG
                    # ,'PPG5' = PPG3, 'PPG10' same as PPG9 ,'PPG26' same as 25
                    # ,'PPG22' morte

# experiments of 07-01-2022, double puce V6, degased medium, no coating with 500ul/heure medium flow
# Stade3 (close to the edge) cups
StackList220107_S3 = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG8','PPG9','PPG10',
                   'PPG11','PPG12','PPG13','PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20']

# experiments of 12-01-2022, double puce V6, degased medium, no coating with 500ul/heure medium flow
# Stade1 (close to the center) cups
StackList220112_S1 = ['PPG1','PPG2','PPG3','PPG5','PPG6','PPG8','PPG9','PPG10','PPG11','PPG13',
                      'PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20','PPG21','PPG22','PPG23','PPG24','PPG25',
                      'PPG26','PPG27','PPG28','PPG29','PPG31','PPG32','PPG33','PPG34','PPG35','PPG36','PPG37',
                      'PPG38','PPG39','PPG40','PPG41','PPG42','PPG43','PPG44','PPG45','PPG46','PPG47','PPG48','PPG49',
                      'PPG50','PPG51','PPG52','PPG53','PPG54','PPG55','PPG56','PPG57','PPG58','PPG59','PPG60','PPG61',
                      'PPG62','PPG63','PPG64','PPG65','PPG66','PPG67','PPG68']
                    # PPG30 erreur de création de la stack #,'PPG7' ppg hors champ image 0
                    # ,'PPG4' same than 1,  'PPG12', same than 8

# experiments of 12-01-2022, double puce V6, degased medium, no coating with 500ul/heure medium flow
# Stade3 (close to the edge) cups
StackList220112_S3 = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG8','PPG9','PPG10','PPG11','PPG12','PPG13',
                      'PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20','PPG21','PPG22','PPG23','PPG24','PPG25',
                      'PPG26','PPG27','PPG28','PPG29','PPG30','PPG31','PPG32','PPG33','PPG34','PPG35','PPG36','PPG37',
                      'PPG38','PPG39','PPG40','PPG41','PPG42','PPG43','PPG44','PPG45','PPG46','PPG47','PPG48','PPG49',
                      'PPG50']
                        

# experiments of 14-01-2022, double puce V6, degased medium, no coating with 500ul/heure medium flow
# Stade1 (close to the center) cups
StackList220114_S1 = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG8','PPG9','PPG10','PPG11','PPG12','PPG13',
                      'PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20','PPG21','PPG22','PPG23','PPG24','PPG25',
                      'PPG26','PPG27','PPG28','PPG29','PPG30','PPG31','PPG32','PPG33','PPG34','PPG35','PPG36','PPG37',
                      'PPG38','PPG39','PPG40','PPG41','PPG42','PPG43','PPG44','PPG45','PPG46','PPG47','PPG48','PPG49',
                      'PPG50','PPG51']
                
# experiments of 14-01-2022, double puce V6, degased medium, no coating with 500ul/heure medium flow
# Stade3 (close to the edge) cups
StackList220114_S3 = ['PPG1','PPG2','PPG3','PPG5','PPG6','PPG7','PPG8','PPG9','PPG10','PPG11','PPG12','PPG13',
                      'PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20','PPG21','PPG22','PPG23','PPG24','PPG25',
                      'PPG26','PPG27','PPG28','PPG29','PPG30','PPG31']
                    # ,'PPG4' same as PPG5
    
    # experiments of 24-01-2022, double puce V6, degased medium, no coating with 500ul/heure medium 
# flow, chip ctrl 1
StackList220124_Ct1 = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG8','PPG9','PPG10','PPG11','PPG12','PPG13',
                      'PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20','PPG21','PPG22','PPG23','PPG24','PPG25',
                      'PPG26','PPG27','PPG28','PPG29','PPG30','PPG31','PPG32','PPG33','PPG34','PPG35']
                
# experiments of 24-01-2022, double puce V6, degased medium, no coating with 500ul/heure medium 
# flow, chip ctrl 2
StackList220124_Ct2 = ['PPG1','PPG2','PPG3','PPG4','PPG5','PPG6','PPG7','PPG8','PPG9','PPG10','PPG11','PPG12','PPG13',
                      'PPG14','PPG15','PPG16','PPG17','PPG18','PPG19','PPG20','PPG21','PPG22','PPG23','PPG24','PPG25',
                      'PPG26','PPG27','PPG28','PPG29','PPG30','PPG31','PPG32','PPG33','PPG34','PPG35','PPG36','PPG37',
                      'PPG38','PPG39','PPG40','PPG41','PPG42','PPG43','PPG44','PPG45','PPG46','PPG47','PPG48','PPG49',
                      'PPG50','PPG51','PPG52','PPG53']
    
print('Data choice made.')


### Test experiments

#### Flux controls V5 (03/27-09-2021)

In [ ]:
# 03-09-2021, V5 flux controls, 500µlh & 1mlh

ParametriseContour('210903_1mlh',P210903_1,'03-09-21 - 1mlh',StackList210903_1,Scale210903,'PR',debug=True)

# ParametriseContour('210903_500ulh',P210903_5,'03-09-21 - 500µlh',StackList210903_5,Scale210903,'P')

# 27-09-2021, V5 flux controls, 500µlh & 1mlh

# ParametriseContour('210927_1mlh',P210927_1,'27-09-21 - 1mlh',StackList210927_1,Scale210927,'P')

# ParametriseContour('210927_500ulh',P210927_5,'27-09-21 - 500µlh',StackList210927_5,Scale210927,'P')

#### Manitol 125mM V5 (14-09-2021)

In [ ]:
# 14-09-2021, V5 manitol 125mM, 500µlh 

# ParametriseContour('210914_Ct',P210914_Ct,'14-09-21 - Manitol Ctrl',StackList210914_Ct,Scale210914,'P')

# ParametriseContour('210914_M125',P210914_M125,'14-09-21 - Manitol 125mM',StackList210914_M125,Scale210914,'P')

#### System control DV6 (22-10-2021)

In [ ]:
# 22-10-2021, DV6 same condition in the two chambers, 500µlh 

# ParametriseContour('211022_Ct1',P211022_Ct1,'22-10-21 - Ctrl1',StackList211022_Ct1,Scale211022,'P')

# ParametriseContour('211022_Ct2',P211022_Ct2,'22-10-21 - Ctrl2',StackList211022_Ct2,Scale211022,'P')


#### Degas & BSA controls DV6 (05-11-2021, 22-12-2021)

In [ ]:
# 05-11-2021, DV6 degased medium, BSA  vs. not coated chambers, 500µlh 

# ParametriseContour('211105_Degas',P211105_Degas,'05-11-21 - Degas ctrl',StackList211105_Degas,Scale211105,'P')

# ParametriseContour('211105_Deg_Bsa',P211105_Deg_Bsa,'05-11-21 - Degas BSA',StackList211105_Deg_Bsa,Scale211105,'P')

# # 22-12-2021, DV6 degased medium, BSA  vs. not coated chambers, 500µlh 

# ParametriseContour('211222_Degas',P211222_Degas,'22-12-21 - Degas ctrl',StackList211222_Degas,Scale211222,'LPR')

# ParametriseContour('211222_Deg_Bsa',P211222_Deg_Bsa,'22-12-21 - Degas BSA',StackList211222_Deg_Bsa,Scale211222,'RP')


### Method paper experiments (Degas medium, Stade1 cups, 500µl/h)

#### Stade comparison DV6 (07/12/14-01-2022)

In [ ]:
# 07-01-2022, DV6 degased medium, Stade1 vs. Stade3 cups, 500µlh 

# ParametriseContour('220107_S1',P220107_S1,'07-01-2022 Stade 1 cups',StackList220107_S1,Scale220107,'LPR',
#                    ldmkplots=True, Overwrite = True, Dmax = 20)

# ParametriseContour('220107_S3',P220107_S3,'07-01-2022 Stade 3 cups',StackList220107_S3,Scale220107,'L',l
#                    dmkplots=True, Overwrite = True, Dmax = 0)

# 12-01-2022, DV6 degased medium, Stade1 vs. Stade3 cups, 500µlh 

# ParametriseContour('220112_S1',P220112_S1,'12-01-2022 Stade 1 cups',StackList220112_S1,Scale220112,'L',
#                    ldmkplots=True, FirstSlice = False, AutoLdmks = False, Overwrite = False)

# ParametriseContour('220112_S3',P220112_S3,'12-01-2022 Stade 3 cups',StackList220112_S3,Scale220112,'L',
#                    ldmkplots=True, FirstSlice = False, AutoLdmks = False, Overwrite = False)

# 14-01-2022, DV6 degased medium, Stade1 vs. Stade3 cups, 500µlh 

# ParametriseContour('220114_S1',P220114_S1,'14-01-2022 Stade 1 cups',StackList220114_S1,Scale220114,'L',
#                    ldmkplots=True, FirstSlice = False, AutoLdmks = False, Overwrite = False)

# ParametriseContour('220114_S3',P220114_S3,'14-01-2022 Stade 3 cups',StackList220114_S3,Scale220114,'L',
#                    ldmkplots=True, FirstSlice = False, AutoLdmks = False, Overwrite = False)

#### System controls DV6 (24-01-2022)

In [ ]:
# ParametriseContour('220124_Ct1',P220124_Ct1,'24-01-2022 Ctrl1',StackList220124_Ct1,Scale220124,'L',
#                    ldmkplots=True, FirstSlice = False, AutoLdmks = False, Overwrite = False)

# ParametriseContour('220124_Ct2',P220124_Ct2,'24-01-2022 Ctrl2',StackList220124_Ct2,Scale220124,'L',
#                    ldmkplots=True, FirstSlice = False, AutoLdmks = False, Overwrite = False)

## Test Zone

In [ ]:
#%run D:/Users/laplaud/Desktop/PostDoc/Code/Marchantia/PropaguleAnalysis/3_ShapeQuantificationNB.ipynb

In [ ]:
a = np.linspace(0,9,10)
print(a)
print(np.roll(a,-4))